In [1]:
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.corpus import wordnet
import torch

In [2]:
rootdir=os.getcwd()+'/coref_data/'
datas={}
spacy.require_cpu()
nlp = spacy.load("en_core_web_trf")
files = [f for f in listdir(rootdir) if isfile(join(rootdir, f))]
for f in files:
    #ff=f[:-4]
    with open(rootdir+f, "r") as file:
        data = file.read().replace("\n", "")
    article = data.replace(u"\xa0", u" ")
    datas[f]=nlp(article)

In [5]:
def list_entities(text):
    ents_=text.ents
    ents=[]
    for x in ents_:
        if x.text not in ents:
            if x.label_=="ORG" or x.label_=="PRODUCT" or x.label_=="GPE":
                ents.append(x.text)
    return ents


def find_org(s, ents):
    i=0
    ind=[]
    for token in s:
        if str(token) in ents: 
            ind.append(i)
        i=i+1
    return ind

def check_synonym_SVO(s, list_):
    for token in s:
        if token.dep_=="ROOT":
            lem=token.lemma_
            if lem in list_:
                return True
            else:
                return False      

def find_sbj(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="nsubj":
            subjects.append(s[i])
        elif s[i].head.dep_=="nsubj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue # could move one more step up the dependency tree if we condition was came up before (excluding object and verbs)
    return subjects

def find_nsubjpass(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="nsubjpass":
            subjects.append(s[i])
        elif s[i].head.dep_=="nsubjpass": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue # could move one more step up the dependency tree if we condition was came up before (excluding object and verbs)
    return subjects
            
def find_obj(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="dobj" or s[i].dep_=="pobj":
            subjects.append(s[i])
        elif s[i].head.dep_=="dobj" or s[i].head.dep_=="pobj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue
    return subjects

def check_synonym_attr(s, list_):
    for token in s:
        if token.dep_=="attr":
            lem=token.lemma_
            if lem in list_:
                return True
            else:
                return False
            
def find_obj_attr(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="pobj":
            subjects.append(s[i])
        elif s[i].head.dep_=="pobj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue
    return subjects

def check_synonym_acl(s, list_):
    for token in s:
        if token.dep_=="acl":
            lem=token.lemma_
            if lem in list_:
                return True
            else:
                return False  

def find_obj_acl(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="pobj":
            subjects.append(s[i])
        elif s[i].head.dep_=="pobj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue
    return subjects

# SVO

Get synonyms

In [6]:
synonyms = []
antonyms = []
  
for syn in wordnet.synsets("supply"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())
print(synonyms)
print(antonyms)

['supply', 'supply', 'provision', 'supply', 'supplying', 'supply', 'provide', 'render', 'furnish', 'issue', 'supply', 'provide', 'supply', 'ply', 'cater', 'add', 'append', 'supply']
['demand', 'recall']


In [7]:
synonyms = []
antonyms = []
  
for syn in wordnet.synsets("provide"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())
print(synonyms)
print(antonyms)

['supply', 'provide', 'render', 'furnish', 'provide', 'supply', 'ply', 'cater', 'provide', 'put_up', 'provide', 'offer', 'leave', 'allow_for', 'allow', 'provide', 'provide', 'bring_home_the_bacon', 'provide']
[]


In [8]:
synonyms = []
antonyms = []
  
for syn in wordnet.synsets("furnish"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())
print(synonyms)
print(antonyms)

['supply', 'provide', 'render', 'furnish', 'furnish']
[]


In [9]:
tuples=[]
relation="suppliers"
synonyms=["supply","provision","provide","furnish","render"] # have is too misleading
for i in files:
    text=datas[i]
    ents=list_entities(text)
    for s in text.sents:
        dict_={}
        org=find_org(s,ents)
        if len(org)>=2:
            SVO=check_synonym_SVO(s,synonyms)
            if SVO==True:
                sbj=find_sbj(s,org)
                if len(sbj)>0: 
                    obj=find_obj(s,org)
                    if len(obj)>0:
                        dict_['relation']=relation
                        ## add ID later dict_['relation']
                        dict_['sentence']=s
                        dict_['entities_A']=sbj
                        dict_['entities_B']=obj
                        tuples.append(dict_)
                else:
                    nsubjpass=find_nsubjpass(s,org)
                    if len(nsubjpass)>0:
                        obj=find_obj(s,org)
                        if len(obj)>0:
                            dict_['relation']=relation
                            ## add ID later dict_['relation']
                            dict_['sentence']=s
                            dict_['entities_A']=obj
                            dict_['entities_B']=sbj
                            tuples.append(dict_)
                    else:
                        continue
            else:
                continue
        else:
            continue    

In [10]:
len(tuples)

2

In [11]:
tuples

[{'relation': 'suppliers',
  'sentence': China's fledgling Cross-Border Interbank Payment System, or CIPS, may provide another alternative to SWIFT.,
  'entities_A': [China],
  'entities_B': [SWIFT]},
 {'relation': 'suppliers',
  'sentence': They will provide on-call support and consultancy for all those travelling to participate in MWC22.Once again, the GSMA has selected bnetwork as the official accommodation agency of MWC22.,
  'entities_A': [GSMA],
  'entities_B': [bnetwork, MWC22]}]

Again, identifies products need to changeidentity NER

# attributes kind of patterns

In [12]:
synonyms = []
antonyms = []
  
for syn in wordnet.synsets("supplier"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())
print(synonyms)
print(antonyms)

['supplier', 'provider']
[]


In [13]:
synonyms = []
antonyms = []
  
for syn in wordnet.synsets("provider"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())
print(synonyms)
print(antonyms)

['supplier', 'provider', 'provider']
[]


In [14]:
tuples=[]
relation="suppliers"  
synonyms=["supplier","provider"]
for i in files:
    text=datas[i]
    ents=list_entities(text)
    for s in text.sents:
        dict_={}
        org=find_org(s,ents)
        if len(org)>=2:
            attr=check_synonym_attr(s,synonyms)
            if attr==True:
                sbj=find_sbj(s,org)
                if len(sbj)>0: 
                    obj=find_obj_attr(s,org)
                    if len(obj)>0:
                        dict_['relation']=relation
                        ## add ID later dict_['relation']
                        dict_['sentence']=s
                        dict_['entities_A']=sbj
                        dict_['entities_B']=obj
                        tuples.append(dict_)

In [15]:
tuples

[{'relation': 'suppliers',
  'sentence': Rimini Street, Inc. (Nasdaq: RMNI) is a global provider of enterprise software products and services, the leading third-party support provider for Oracle and SAP software products and a Salesforce partner.,
  'entities_A': [Nasdaq],
  'entities_B': [Oracle]},
 {'relation': 'suppliers',
  'sentence': This would "shatter any EU illusions that Russia is a reliable supplier" and would likely trigger a "concerted effort" within the bloc to "permanently reduce gas imports from Russia as soon as possible," he added.,
  'entities_A': [Russia],
  'entities_B': [Russia]}]

In [16]:
# nothing relevant

# ACL based

In [17]:
tuples=[]
relation="suppliers"
synonyms=["supply","provision","provide","furnish","render"] # have is too misleading
for i in files:
    text=datas[i]
    ents=list_entities(text)
    for s in text.sents:
        dict_={}
        org=find_org(s,ents)
        if len(org)>=2:
            acl=check_synonym_acl(s,synonyms)
            if acl==True:
                sbj=find_sbj(s,org)
                if len(sbj)>0: 
                    obj=find_obj_acl(s,org)
                    if len(obj)>0:
                        dict_['relation']=relation
                        ## add ID later dict_['relation']
                        dict_['sentence']=s
                        dict_['entities_A']=sbj
                        dict_['entities_B']=obj
                        tuples.append(dict_)

In [18]:
tuples

[]

In [19]:
# same use ORG although nothing comes up with ORG